# Выбор локации для скважины

Выбор локации для скважины добывающей компании «ГлавРосГосНефть». 

Предоставлены пробы нефти в трёх регионах: в каждом 10 000 месторождений, где измерили качество нефти и объём её запасов. Нам предстоит модель машинного обучения, которая поможет определить регион, где добыча принесёт наибольшую прибыль. А также проанализировать возможную прибыль и риски техникой *Bootstrap.*

Шаги для выбора локации:

- В избранном регионе найдем месторождения, для каждого определим значения признаков;
- Постороим модель и оценим объём запасов;
- Выбирем месторождения с самыми высокими оценками значений. Количество месторождений зависит от бюджета компании и стоимости разработки одной скважины;
- Прибыль равна суммарной прибыли отобранных месторождений.

## Загрузка и подготовка данных

Загружаем библиотеки, сохраняем данные в переменных, знакомимся с данными

In [26]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

state = np.random.RandomState(42)

data_0 = pd.read_csv('/datasets/geo_data_0.csv')
data_1 = pd.read_csv('/datasets/geo_data_1.csv')
data_2 = pd.read_csv('/datasets/geo_data_2.csv')

display(data_0.head())
display(data_1.head())
display(data_2.head())

,id,f0,f1,f2,product
0,txEyH,0.705745,-0.497823,1.221170,105.280062
1,2acmU,1.334711,-0.340164,4.365080,73.037750
2,409Wp,1.022732,0.151990,1.419926,85.265647
3,iJLyR,-0.032172,0.139033,2.978566,168.620776
4,Xdl7t,1.988431,0.155413,4.751769,154.036647


,id,f0,f1,f2,product
0,kBEdx,-15.001348,-8.276000,-0.005876,3.179103
1,62mP7,14.272088,-3.475083,0.999183,26.953261
2,vyE1P,6.263187,-5.948386,5.001160,134.766305
3,KcrkZ,-13.081196,-11.506057,4.999415,137.945408
4,AHL4O,12.702195,-8.147433,5.004363,134.766305


,id,f0,f1,f2,product
0,fwXo0,-1.146987,0.963328,-0.828965,27.758673
1,WJtFt,0.262778,0.269839,-2.530187,56.069697
2,ovLUW,0.194587,0.289035,-5.586433,62.871910
3,q6cA6,2.236060,-0.553760,0.930038,114.572842
4,WPMUX,-0.515993,1.716266,5.899011,149.600746


Проверяем на наличие пропусков.

In [27]:
display(data_0.isna().sum())
display(data_1.isna().sum())
display(data_2.isna().sum())


id         0
f0         0
f1         0
f2         0
product    0
dtype: int64

id         0
f0         0
f1         0
f2         0
product    0
dtype: int64

id         0
f0         0
f1         0
f2         0
product    0
dtype: int64

Разделим данные на обучающие и валидационные выборки:

In [28]:
features_0, features_1, features_2 = (
    data_0.drop(['id', 'product'], axis=1),
    data_1.drop(['id', 'product'], axis=1),
    data_2.drop(['id', 'product'], axis=1)
)
target_0, target_1, target_2 = data_0['product'], data_1['product'], data_2['product']

display(features_0.shape)
display(target_0.shape)
display(features_1.shape)
display(target_1.shape)
display(features_2.shape)
display(target_2.shape)

(100000, 3)

(100000,)

(100000, 3)

(100000,)

(100000, 3)

(100000,)

Так как столбец id не поможет нам улучшить качество моделей - исключим его из обучающих данных.

In [29]:
features_train_0, features_valid_0, target_train_0, target_valid_0 = train_test_split(
    features_0, target_0, test_size=0.25, random_state=state)

features_train_1, features_valid_1, target_train_1, target_valid_1 = train_test_split(
    features_1, target_1, test_size=0.25, random_state=state)

features_train_2, features_valid_2, target_train_2, target_valid_2 = train_test_split(
    features_2, target_2, test_size=0.25, random_state=state)

display(features_train_0.shape)
display(features_valid_0.shape)
display(target_train_0.shape)
display(target_valid_0.shape)
display(features_train_1.shape)
display(features_valid_1.shape)
display(target_train_1.shape)
display(target_valid_1.shape)
display(features_train_2.shape)
display(features_valid_2.shape)
display(target_train_2.shape)
display(target_valid_2.shape)


(75000, 3)

(25000, 3)

(75000,)

(25000,)

(75000, 3)

...

(25000,)

(75000, 3)

(25000, 3)

(75000,)

(25000,)

Разобьем данные на обучающие и валидационные выборки, проверим с помощью функции shape.

In [30]:
pd.options.mode.chained_assignment = None

from sklearn.preprocessing import StandardScaler

def super_scaler(data_train, data_valid):
    numeric = data_train.columns

    scaler = StandardScaler()
    scaler.fit(data_train[numeric])
    data_train[numeric] = scaler.transform(data_train[numeric])
    data_valid[numeric] = scaler.transform(data_valid[numeric])
    return data_train, data_valid

features_train_0, features_valid_0 = super_scaler(features_train_0, features_valid_0)
features_train_1, features_valid_1 = super_scaler(features_train_1, features_valid_1)
features_train_2, features_valid_2 = super_scaler(features_train_2, features_valid_2)

features_train_0.head()


,f0,f1,f2
98980,1.274786,-0.799739,-0.396677
69824,-1.600689,0.234678,-2.169283
9928,-0.323791,1.436297,1.495425
75599,0.439038,0.830679,0.185881
95621,-1.652805,0.761012,0.111734


Создал функцию для масштабирования обучающих данных. Качество предсказаний моделей это не улучшило. Для линейной регрессии этого не надо делать?

## Обучение и проверка модели

Создадим функцию для обучения модели логистической регрессии с расчетеом RMSE и среднего запаса сырья в регионе, предсказания и ответы сохраним в переменных. 
Вопрос: почему по условиям задачи именно логистическая регрессия (я почитал еще про Ridge и ElasticNet), и почему рассчитываем именно RMSE? Так как это первый проект, где надо применять методы регресии, не очень понимаю метрик регрессии, может есть какие-нибудь статьи по выбору метрик и их интерпретации?

In [31]:
def model_predict(features_train, target_train, features_valid, target_valid):
    model = LinearRegression()
    model.fit(features_train, target_train)
    predictions_valid = model.predict(features_valid)
    mean = predictions_valid.mean()
    rmse = (mean_squared_error(target_valid, predictions_valid))**0.5
    print('Средний запас =', round(mean, 2))
    print('RMSE модели  =', round(rmse, 2))
    return predictions_valid, target_valid



In [32]:
predictions_valid_0, target_valid_0 = model_predict(features_train_0, target_train_0, features_valid_0, target_valid_0)

Средний запас = 92.4
RMSE модели  = 37.76


In [33]:
predictions_valid_1, target_valid_1 = model_predict(features_train_1, target_train_1, features_valid_1, target_valid_1)

Средний запас = 68.59
RMSE модели  = 0.89


In [34]:
predictions_valid_2, target_valid_2 = model_predict(features_train_2, target_train_2, features_valid_2, target_valid_2)

Средний запас = 95.09
RMSE модели  = 40.24


Самый низкий запас сырья в регионе 1, качество предсказаний самое лучшее так же в 1 регионе.

Попытался применить GridSearch и кроссвалидацию на линейной регресии, протратил много времени, результат хуже, чем без них.
Для линейной регресии в этом нет смысла? или я что-то не так сделал? и почему у меня RMSE получися отрицательным в обоих случаях?

In [35]:
from sklearn.model_selection import GridSearchCV


def models_rmse(features_train, target_train):

    model = LinearRegression()
    parameters = {'fit_intercept':[True,False], 'normalize':[True,False], 'copy_X':[True, False]}
    grid = GridSearchCV(model,parameters, scoring = 'neg_root_mean_squared_error', cv=5)
    grid.fit(features_train, target_train)
    return "rmse : ", grid.best_score_

print(models_rmse(features_train_0, target_train_0))    

('rmse : ', -37.67200536313506)


In [36]:
from sklearn.model_selection import cross_val_score

def get_cv_scores(model, features_train, target_train): 
    scores = cross_val_score(model, 
                             features_train, 
                             target_train, 
                             cv=5, 
                             scoring='neg_root_mean_squared_error') 
    
    model.fit(features_train, target_train)
    print('rmse ', np.mean(scores)) 

lr = LinearRegression()
get_cv_scores(lr, features_train_0, target_train_0)
 

rmse  -37.67200536313506


## Подготовка к расчёту прибыли

Рассчитаем достаточный объём сырья для безубыточной разработки новой скважины. Для этого бюджет разделим на произведение кол-ва точек добычи и ценой за баррель.

In [37]:
budget = 10_000_000_000
best_points = 200
barrels_price = 450_000
print('Достаточный объем сырья =', round(budget/(best_points*barrels_price), 2))

Достаточный объем сырья = 111.11


## Расчёт прибыли и рисков 

Напишем функцию для расчёта прибыли по выбранным скважинам и предсказаниям модели:
-отсортируем предсказания по убыванию.
-выберем заданное количество ответов с наивысшим показателем сырья
-вычтем из бюджета сумму валовой прибыли для оценки чистой прибыли

In [38]:
def profit(target_valid, predictions_valid, count):
    predictions = pd.Series(predictions_valid, index=target_valid.index)
    probs_sorted = predictions.sort_values(ascending=False)
    selected = target_valid[probs_sorted.index][:count]
    return barrels_price * selected.sum() - budget

In [39]:
print('Прибыль в регионе 0 =',round(profit(target_valid_0, predictions_valid_0, 200)/1000000000, 2), 'млрд')

Прибыль в регионе 0 = 3.36 млрд


In [40]:
print('Прибыль в регионе 1 =',round(profit(target_valid_1, predictions_valid_1, 200)/1000000000, 2), 'млрд')

Прибыль в регионе 1 = 2.42 млрд


In [41]:
print('Прибыль в регионе 2 =',round(profit(target_valid_2, predictions_valid_2, 200)/1000000000, 2), 'млрд')

Прибыль в регионе 2 = 2.58 млрд


Далее напишем функцию для расчета средней прибыли для региона, 95% доберительного интервала и риска убытков:
-для лучшей визуализации удем делить результаты на миллион и округлять до 2 знаков после запятой.
-с помощью бутстрепа 1000 раз выборем случайные 500 точек добычи, из них выберем 200 наибольших по запасу сырья с помощью сортировки.
-Расчитаем среднюю прибыли=ь для региона, 95% доберительный интервал(по 2.5% с каждой стороны) и риск убытков, т.е. pvalue:

In [42]:
million = 1000000
def risk_profit(target_valid, predictions_valid, name):
    values = []
    for i in range(1000):
        target_subsample = target_valid.sample(500, replace=True, random_state=state)
        probabilities = pd.Series(predictions_valid, index=target_valid.index)
        probs_subsample = probabilities[target_subsample.index]
        target_subsample = target_subsample.reset_index(drop = True)
        probs_subsample = probs_subsample.reset_index(drop = True)
        values.append(profit(target_subsample, probs_subsample, 200))
    values = pd.Series(values)
    mean = values.mean()/million
    lower = values.quantile(0.025)/million
    upper = values.quantile(0.975)/million
    print('Средняя прибыль для региона', name, '=', mean, 'млн')
    print('95%-й доверительный интервал для региона', name, '=', 'от', round(lower, 2), 'млн',
          'до', round(upper, 2), 'млн')
    print('Риск убытков =', (values < 0).mean())

Далее сохраним в регионы, целевые значения и предсказания в списки для визуализации через функции через цикл:

In [43]:
region = ['0', '1', '2']
target = [target_valid_0,  target_valid_1, target_valid_2]
predictions = [predictions_valid_0, predictions_valid_1, predictions_valid_2]
for region, target, predictions in zip(region, target, predictions):
    risk_profit(target, predictions, region)
    print()

Средняя прибыль для региона 0 = 415.6606390640568 млн
95%-й доверительный интервал для региона 0 = от -136.33 млн до 932.34 млн
Риск убытков = 0.062

Средняя прибыль для региона 1 = 429.5446180738954 млн
95%-й доверительный интервал для региона 1 = от 37.48 млн до 813.8 млн
Риск убытков = 0.015

Средняя прибыль для региона 2 = 318.1530922311814 млн
95%-й доверительный интервал для региона 2 = от -227.79 млн до 813.88 млн
Риск убытков = 0.123



# Вывод:

Так как в регионе под номером 1  наивысший показатель средней прибыли, наименьший риск убытков,
а максимальная прибыль не сильно ниже чем в регионе под номером 0- рекомендовано разрабатывать регион под номером 1.